In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df1 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/DRF-64 - Brandon Myers Special Education 2025 December Count 01232025.xlsx")

In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200529 entries, 0 to 200528
Data columns (total 23 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   SchoolYear                  200529 non-null  int64  
 1   SubmissionWindowId          200529 non-null  int64  
 2   REPORTING_CORPORATION       199637 non-null  float64
 3   REPORTING_CORPORATION_NAME  199637 non-null  object 
 4   LEGAL_CORPORATION           188030 non-null  float64
 5   LEGAL_CORPORATION_NAME      188030 non-null  object 
 6   IDOE_SCHOOL_ID              200529 non-null  object 
 7   SCHOOL_NAME                 200529 non-null  object 
 8   STN                         200529 non-null  object 
 9   GENDER                      200529 non-null  object 
 10  ETHNICITY                   200529 non-null  object 
 11  BIRTH_DATE                  200529 non-null  object 
 12  AGE_ON_01_DEC_2024          200529 non-null  int64  
 13  Grade         

In [4]:
#df1.nunique()

In [5]:
df1.columns = df1.columns.str.lower()

In [6]:
duplicated_stn = df1.duplicated(keep = False)


In [7]:
duplicated_stn.sum()

np.int64(28)

In [8]:
df1_sorted  = pd.concat([df1[duplicated_stn],df1[~duplicated_stn]])

In [9]:
#df1_sorted.head(5)

In [10]:
df1_sorted.drop_duplicates(inplace = True)

In [11]:
df1_sorted.shape

(200515, 23)

In [12]:
df1_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200515 entries, 14465 to 200528
Data columns (total 23 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   schoolyear                  200515 non-null  int64  
 1   submissionwindowid          200515 non-null  int64  
 2   reporting_corporation       199623 non-null  float64
 3   reporting_corporation_name  199623 non-null  object 
 4   legal_corporation           188016 non-null  float64
 5   legal_corporation_name      188016 non-null  object 
 6   idoe_school_id              200515 non-null  object 
 7   school_name                 200515 non-null  object 
 8   stn                         200515 non-null  object 
 9   gender                      200515 non-null  object 
 10  ethnicity                   200515 non-null  object 
 11  birth_date                  200515 non-null  object 
 12  age_on_01_dec_2024          200515 non-null  int64  
 13  grade          

In [13]:
#df1_sorted.grade.value_counts()

####Replace KG - Morning to KG

In [14]:
df1_sorted.replace('KG - Morning', 'KG', inplace = True)

In [15]:
# columns convert float to integer
cols = ['reporting_corporation', 'legal_corporation', 'primaryexceptionality', 'secondaryexceptionality', 'location', 'adm_type_id']
df1_sorted[cols] = df1_sorted[cols].astype('Int64')

In [16]:
df1_sorted['grade'] = df1_sorted['grade'].astype(str)

In [17]:
df1_sorted['grade'] = df1_sorted['grade'].str.strip()

In [18]:
df1_sorted.location.value_counts()

,count
location,
10,8656
3,1221
4,227
9,121
2,64
6,7


In [19]:
# to display all columns
pd.set_option("display.max_columns", None)

#### ProgramName Extraction based on the data requirment

In [20]:
#df1_sorted.programname.value_counts()

####Excluded from the above list

*   Choice Voucher
*   Identified Unserved NonPublic/Homeschool Only
*   Identified Unserved NonPublic/HomeSchool Only
*   Correctional Facility
*   Parentally placed in Private School





In [21]:
values_to_remove = ['Choice Voucher',
                    'Identified Unserved NonPublic/Homeschool Only',
                    'Identified Unserved NonPublic/HomeSchool Only',
                    'Correctional Facility',
                    'Parentally placed in Private School'
]

df1_sorted = df1_sorted[~df1_sorted['programname'].isin(values_to_remove)]

In [22]:
#df1_sorted.programname.value_counts() # check the values

######Replacing Ethnicity based on the fedral values

In [23]:
df1_sorted.replace({'White':'WH7', 'Black':'BL7', 'Hispanic': 'HI7', 'Multiracial': 'MU7', 'Asian' : 'AS7', 'American Indian' : 'AM7', 'Native Hawaiian or Other Pacific Islander' : 'PI7'}, inplace = True)

In [24]:
# check the ethnicity values
#df1_sorted.ethnicity.value_counts()

In [25]:
df1_sorted.location.value_counts()

,count
location,
10,971
9,121
2,64
4,16
6,7


####Exclude location: 2, 3, 4, 10

In [26]:
df1_sorted = df1_sorted[~df1_sorted['location'].isin([2, 3, 4, 10])]

In [27]:
df1_sorted.location.value_counts()

,count
location,
9,121
6,7


##All children with disability from age 3-21

In [28]:
age_3_21 = df1_sorted.query("age_on_01_dec_2024 >= 3 & age_on_01_dec_2024 <= 21")

In [29]:
#age_3_21.age_on_01_dec_2024.value_counts()# check the validity

####Master corp number from Sig Dispro file

In [30]:
master_corp = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/corp_no_from_sig_dispro_calc.xlsx")

In [31]:
master_corp.head()

,Reporting Corp
0,-999
1,15
2,25
3,35
4,125


In [32]:
all_corps = master_corp['Reporting Corp'].unique()

## All children with disabiity for IDEA-Identification sheet age 3-21

In [33]:
all_children_with_disability_3_21  = age_3_21.groupby('reporting_corporation')['ethnicity'].value_counts().unstack(fill_value=0).sort_values(by = 'reporting_corporation')


In [34]:
#Save the file
all_children_with_disability_3_21.to_csv('all_children_with_disability-IDEA_Identification_3_21.csv')
all_children_with_disability_3_21_corp = all_children_with_disability_3_21.reindex(all_corps, fill_value = 0) # remove the hash sign for future download
#all_children_with_disability_3_21_corp

In [35]:
all_children_with_disability_3_21_corp.to_excel('all_children_with_disability_3_21_corp.xlsx')

In [36]:
all_corps.shape

(511,)

#####merging the disability file to extract disability name from the disability code

In [37]:
disability = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Disability_codes.xlsx")

In [38]:
disability.head(10)

,Code,Disability
0,1,Multiple Disabilities
1,2,Orthopedic Impairment
2,3,Blind or Low Vision
3,4,Deaf or Hard of Hearing
4,5,Emotional Disability (Full-Time)
5,6,Emotional Disability (Part-Time)
6,7,Specific Learning Disability
7,8,Developmental Delay
8,9,Speech/Language Impairment
9,10,Mild Intellectual Disability


In [39]:
clean_df_merge = age_3_21.merge(disability, left_on = 'primaryexceptionality', right_on = 'Code', how = 'left')

In [40]:
#clean_df_merge.tail() # validated the exact disability code, use both tail and head macth with disability file
#Remove hash to run the code

#####Combine Emotional Disability(part time) to Emotional Disability(full time)- To do  that replace part time disability to full time disability
and replce Emotional disability(full-time) to Emotional disability

In [41]:
#Replace part time disability to full time disability, and replce Emotional disability(full-time) to Emotional disability
clean_df_merge.replace({'Emotional Disability (Part-Time)': 'Emotional Disability (Full-Time)'}, inplace= True)
clean_df_merge.replace({'Emotional Disability (Full-Time)': 'Emotional Disability'}, inplace= True)

##### Combine Mild Intellectual Disability, Moderate Intellectual Disability and Severe Intellectual Disability. to to that replace all with Intelllectual disability

In [42]:
#Replace Mild Intellectual Disability, Moderate Intellectual Disability and Severe Intellectual Disability to Intellectual disability

clean_df_merge.replace({'Mild Intellectual Disability': 'Intellectual Disability', 'Moderate Intellectual Disability': 'Intellectual Disability', 'Severe Intellectual Disability': 'Intellectual Disability'}, inplace= True)

In [43]:
#clean_df_merge.Disability.value_counts() # validate the exact replacement

###Filter Autism for all races

In [44]:
all_races_autism = clean_df_merge.query("Disability == 'Autism Spectrum Disorder'")

In [45]:
all_races_autism.Disability.value_counts() # Validate the exact filter

,count
Disability,
Autism Spectrum Disorder,21493


In [46]:
identification_autism = all_races_autism.groupby('reporting_corporation')['ethnicity'].value_counts().unstack(fill_value=0).sort_values(by = 'reporting_corporation').reindex(all_corps, fill_value = 0)

In [47]:

#Save the file
identification_autism.to_excel('identification_autism-3_21.xlsx') # remove the hash sign for future download
#identification_autism


##### Filter Intellectual disability

In [48]:
intellectual_disability = clean_df_merge.query("Disability == 'Intellectual Disability'")
intellectual_disability.Disability.value_counts()

,count
Disability,
Intellectual Disability,12257


In [49]:
identification_intellectual_disbility = intellectual_disability.groupby('reporting_corporation')['ethnicity'].value_counts().unstack(fill_value=0).sort_values(by = 'reporting_corporation').reindex(all_corps, fill_value = 0)

In [50]:

#Save the file
identification_intellectual_disbility.to_excel('identification_intellectual_disbility_3-21.xlsx') # remove the hash sign for future download
#identification_intellectual_disbility

##### Filter Emotional Disability

In [51]:
emotional_disability = clean_df_merge.query("Disability == 'Emotional Disability'")
emotional_disability.Disability.value_counts()


,count
Disability,
Emotional Disability,10803


In [52]:
emotional_disability_identification = emotional_disability.groupby('reporting_corporation')['ethnicity'].value_counts().unstack(fill_value=0).sort_values(by = 'reporting_corporation').reindex(all_corps, fill_value = 0)

In [53]:
#save the file
emotional_disability_identification.to_excel('emotional_disability_identification_3_21.xlsx') # remove the hash sign for future download
#emotional_disability_identification

##### Filter Other Health Impairment




In [54]:
other_health_impaiment = clean_df_merge.query("Disability == 'Other Health Impairment'")
other_health_impaiment.Disability.value_counts()

,count
Disability,
Other Health Impairment,31626


In [55]:
other_health_impaiment_identification = other_health_impaiment.groupby('reporting_corporation')['ethnicity'].value_counts().unstack(fill_value=0).sort_values(by = 'reporting_corporation').reindex(all_corps, fill_value = 0)

In [56]:
other_health_impaiment_identification
#Save the file
other_health_impaiment_identification.to_excel('other_health_impaiment_identification_3_21.xlsx') # remove the hash sign for future download
#other_health_impaiment_identification

####Filter specific learning  disability

In [57]:
specific_learning_disability = clean_df_merge.query("Disability == 'Specific Learning Disability'")
specific_learning_disability.Disability.value_counts()

,count
Disability,
Specific Learning Disability,51767


In [58]:
specific_learning_disability_identification = specific_learning_disability.groupby('reporting_corporation')['ethnicity'].value_counts().unstack(fill_value=0).sort_values(by = 'reporting_corporation').reindex(all_corps, fill_value = 0)

In [59]:
#Save the file
specific_learning_disability_identification.to_excel('specific_learning_disability_identification_3_21.xlsx') # remove the hash sign for future download
#specific_learning_disability_identification


#####Filter Speech or language Impairments

In [60]:
speech_language_impairments = clean_df_merge.query("Disability == 'Speech/Language Impairment'")
speech_language_impairments.Disability.value_counts()


,count
Disability,
Speech/Language Impairment,36855


In [61]:
speech_language_impairments_identification = speech_language_impairments.groupby('reporting_corporation')['ethnicity'].value_counts().unstack(fill_value=0).sort_values(by = 'reporting_corporation').reindex(all_corps, fill_value = 0)

In [62]:
#Save the file
speech_language_impairments_identification.to_excel('speech_language_impairments_identification-3_21.xlsx') # remove the hash sign for future download
#speech_language_impairments_identification


#IDEA Placement age 5- 21 or school age, and grade KG - 12


In [63]:

age_K_12 = df1_sorted[
    (df1_sorted['grade'].isin(['KG', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])) &
    (df1_sorted['age_on_01_dec_2024'] >= 5) &
    (df1_sorted['age_on_01_dec_2024'] <= 21)
]


In [64]:
#age_K_12.grade.value_counts()

In [65]:
#ReRegular classroom (80% or more) replace with Classroom
age_K_12.replace({'Regular classroom (80% or more)': 'Regular Classroom (80% or more)'}, inplace = True)

<ipython-input-65-29d27aa17ae4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_K_12.replace({'Regular classroom (80% or more)': 'Regular Classroom (80% or more)'}, inplace = True)


In [66]:
#age_K_12.programname.value_counts()

####Map with this
Inside a regular class less than 40 of the day(school age) = Seprate Class (less than 40%)

Inside seprate scholls and residential facilies = Seprate Day School Facility(+50%) & Student's disability requires home services

## filter Inside a regular class less than 40%

In [67]:
age_K_12_regular_class = age_K_12.query("programname == 'Separate Class (less than 40%)'")

In [68]:
age_K_12_regular_class.programname.value_counts()

,count
programname,
Separate Class (less than 40%),15421


In [69]:
age_K_12_regular_class_less_than_40 = age_K_12_regular_class.groupby('reporting_corporation')['ethnicity'].value_counts().unstack(fill_value=0).sort_values(by = 'reporting_corporation').reindex(all_corps, fill_value = 0)

In [70]:
#save the file, remvoe the hash for futur download
age_K_12_regular_class_less_than_40.to_excel('age_K_12_regular_class_less_than_40.xlsx')
#age_K_12_regular_class_less_than_40

###Map And filter
Inside seprate schools and residential facilies =

Seprate Day School Facility(+50%) &

Student's disability requires home services &

 Residential Facility (+50%)

In [71]:
age_K_12.programname.value_counts()

,count
programname,
Regular Classroom (80% or more),142255
Separate Class (less than 40%),15421
Resource Room (40% - 79%),13675
Separate Day School Facility (+50%),1390
Student's disability requires home services,1176
Residential Facility (+50%),300


In [72]:
inside_separate = age_K_12[age_K_12['programname'].isin(["Separate Day School Facility (+50%)", "Student's disability requires home services", "Residential Facility (+50%)"])]

In [73]:
#inside_separate.programname.value_counts()

In [74]:
inside_sept_res_facility = inside_separate.groupby('reporting_corporation')['ethnicity'].value_counts().unstack(fill_value=0).sort_values(by = 'reporting_corporation').reindex(all_corps, fill_value = 0)


In [75]:
#save the file
inside_sept_res_facility.to_excel('inside_sept_res_facility.xlsx')
#inside_sept_res_facility
